In [7]:
import yfinance as yf
import pandas as pd

def process_leveraged_data(tickers, leverage_scalar=None):
    """
    Download historical data for given tickers and simulate leveraged ETF data.
    Formats the output DataFrame to include separate columns for each ticker's data.

    Parameters:
        tickers (list of str): List of ticker symbols (e.g., ["QQQ", "SPY"]).
        leverage_scalar (float, optional): Scalar for leveraged equity returns.
            If None, prompts the user for input.

    Returns:
        pd.DataFrame: DataFrame with formatted columns for each ticker.
    """
    result_df = pd.DataFrame()  # Initialize an empty DataFrame for results

    for ticker in tickers:
        print(f"Downloading data for {ticker}...")

        # Download the baseline data
        baseline_data = yf.download(ticker, progress=False)

        # Get leverage scalar from user if not provided
        if leverage_scalar is None:
            while True:
                try:
                    leverage_scalar = float(input(f"Enter the scalar for the leveraged equity returns for {ticker} (e.g., 3): "))
                    break
                except ValueError:
                    print("Invalid input. Please enter a numerical value.")

        # Calculate daily returns
        baseline_data['Daily Return'] = baseline_data['Adj Close'].pct_change()

        # Simulate leveraged returns
        baseline_data['Leveraged Return'] = baseline_data['Daily Return'] * leverage_scalar
        baseline_data.loc[baseline_data.index[0], 'Leveraged Return'] = 0  # Ensure the first leveraged return is 0

        # Calculate simulated leveraged price
        baseline_data['Simulated Leveraged Price'] = (1 + baseline_data['Leveraged Return']).cumprod()
        baseline_data['Simulated Leveraged Price'] *= baseline_data['Adj Close'].iloc[0]  # Normalize to starting price

        # Rename columns to include the ticker symbol
        baseline_data = baseline_data.rename(
            columns={
                'Daily Return': f'DailyReturn_{ticker}',
                'Adj Close': f'AdjClose_{ticker}',
                'Leveraged Return': f'LeveragedReturn_{ticker}',
                'Simulated Leveraged Price': f'SimulatedLeveragedPrice_{ticker}'
            }
        )

        # Keep only relevant columns and reset index
        baseline_data = baseline_data[[f'DailyReturn_{ticker}', f'AdjClose_{ticker}', 
                                       f'LeveragedReturn_{ticker}', f'SimulatedLeveragedPrice_{ticker}']]
        baseline_data.reset_index(inplace=True)

        # Merge with the result DataFrame
        if result_df.empty:
            result_df = baseline_data
        else:
            result_df = pd.merge(result_df, baseline_data, on='Date', how='outer')

    return result_df

In [8]:
data = process_leveraged_data(['QQQ','SPY'], 3)
data.head()

,Date,DailyReturn_QQQ,AdjClose_QQQ,LeveragedReturn_QQQ,SimulatedLeveragedPrice_QQQ,DailyReturn_SPY,AdjClose_SPY,LeveragedReturn_SPY,SimulatedLeveragedPrice_SPY
0,1993-01-29,NaN,NaN,NaN,NaN,NaN,24.526085,0.000000,24.526085
1,1993-02-01,NaN,NaN,NaN,NaN,0.007112,24.700502,0.021335,25.049337
2,1993-02-02,NaN,NaN,NaN,NaN,0.002120,24.752859,0.006359,25.208626
3,1993-02-03,NaN,NaN,NaN,NaN,0.010570,25.014500,0.031710,26.008000
4,1993-02-04,NaN,NaN,NaN,NaN,0.004184,25.119160,0.012552,26.334450
